翻訳器の作成．
英⇒日はwebから引用したものを利用.
日⇒英は100本ノック98で作ったものを利用する.

まずは必要なデータやライブラリをそろえます．

参考 
https://qiita.com/nymwa/items/2f39a34982aa9d71f10d  
https://gist.github.com/nymwa/d584be8ed36b9513a31e31f5b8f50112  
https://qiita.com/O-Kazu-O/items/5c0781913a1d033bea39

## ここから英→日の訓練　訓練は別サーバで行った

In [ ]:
!git clone https://github.com/nymwa/light_enja2.git

Cloning into 'light_enja2'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 4), reused 30 (delta 4), pack-reused 0
Unpacking objects: 100% (30/30), done.
  Using cached sacremoses-0.0.35.tar.gz (859 kB)
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
     |████████████████████████████████| 1.7 MB 55.8 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.2 MB/s 
     |████████████████████████████████| 145 kB 61.7 MB/s 
     |████████████████████████████████| 74 kB 3.6 MB/s 
     |████████████████████████████████| 112 kB 58.1 MB/s 
     |████████████████████████████████| 596 kB 55.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=ac61d11f7a17cc2ea045169675ecba16ee948fc80feb18ef6de7419cae2213c1
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e2

In [ ]:
!mv light_enja2/* .

In [4]:
!pip install -r requirements.txt

  Using cached fairseq-0.10.2.tar.gz (938 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\local\Anaconda3\python.exe' 'C:\local\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' get_requires_for_build_wheel 'C:\Users\ogasa\AppData\Local\Temp\tmpjp5vbsqx'
       cwd: C:\Users\ogasa\AppData\Local\Temp\pip-install-7kuvmah4\fairseq
  Complete output (31 lines):
  Traceback (most recent call last):
    File "setup.py", line 214, in <module>
      do_setup(package_data)
    File "setup.py", line 136, in do_setup
      setup(
    File "C:\Users\ogasa\AppData\Local\Temp\pip-build-env-gtdq9seq\overlay\Lib\site-packages\setuptools\__init__.py", line 154, in setup
      _install_setup_requires(attrs)
    File "C:\Users\ogasa\AppData\Local\Temp\pip-build-env-gtdq9seq\overlay\Lib\site-packages\setuptools\__init__.py", line 148, in _install_setup_requires
      dist.fetch_build_eggs(dist.setup_requires)
    File "C:\Users\ogasa\AppData\Local\Temp\pip-build-env-gtdq9seq\overlay\Lib\site-packages\setuptools\build

データに対して前処理を行います

ダウンロードした時点でデータセットは単語分割やトークン化がすでに施されています．
マージ数8000のBPEでサブワードに分割します．

In [ ]:
! bash preproc.sh

+ TRAIN_EN=corpus/train.en
+ TRAIN_JA=corpus/train.ja
+ VALID_EN=corpus/valid.en
+ VALID_JA=corpus/valid.ja
+ TEST_EN=corpus/test.en
+ cat corpus/train.en corpus/train.ja
+ python src/learn.py --input train.enja --prefix bpe --vocab-size 4000 --character-coverage 0.9995 --threads 1
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: train.enja
  input_format: 
  model_prefix: bpe
  model_type: BPE
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 1
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_voc

学習を行います．

少ないデータでかつかなり軽量なモデルでの学習ですが，10分ぐらいかかります．

In [ ]:
!fairseq-train \
    data-bin \
    --save-interval 10 \
    --max-epoch 10 \
    --update-freq 1 \
    --max-tokens 4000 \
    --arch transformer \
    --encoder-normalize-before \
    --decoder-normalize-before \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 1024 \
    --encoder-attention-heads 4 \
    --encoder-layers 4 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 1024 \
    --decoder-attention-heads 4 \
    --decoder-layers 4 \
    --share-all-embeddings \
    --dropout 0.3 \
    --optimizer adam \
    --adam-betas '(0.9, 0.999)' \
    --lr 0.002 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 2000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 1.0 \
    --weight-decay 0.01 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.6

2022-02-27 16:10:16 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=4, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=4, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=True, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_bac

テストデータで翻訳を行い，その性能を評価します．

評価尺度BLEUで30ぐらい出ると思います．

In [ ]:
! fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint10.pt \
    --beam 5 \
    --lenpen 0.6 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu corpus/test.ja

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 30.6 66.3/41.9/27.1/18.6 (BP = 0.891 ratio = 0.896 hyp_len = 45908 ref_len = 51230)


翻訳結果と正解データの冒頭10文です．

In [ ]:
! head output.txt corpus/test.ja

==> output.txt <==
彼女 の 母 は 美しい 女性 です 。
畑 に は 6 人 の 羊 が い た 。
今朝 は 散歩 の 間 に 大きな 写真 を 撮る こと が でき た 。
今晩 電話 し ます 。
彼女 は 本 の 虫 だ 。
式 は 終わっ た 。
私 は 彼 に 前日 自分 で 楽しん だ か どう か 尋ね た 。
彼 は 注意 し て い ない 。
列車 は いつも 定刻 に いる 。
2 時 に 出発 すれ ば 、 彼 ら は 6 時 に 着く べき だ 。

==> corpus/test.ja <==
彼女 の お母さん は きれい な 女 の 人 です 。
野原 に は 六 頭 の 羊 が い た 。
この 朝 の 散歩 で とても 素晴らしい 写真 を 撮る こと が でき た 。
今夜 電話 し ます 。
彼女 は いわゆる 本 の 虫 です 。
式典 は 終わり まし た 。
昨日 は 楽しかっ た か 、 と 私 は 彼 に 尋ね た 。
彼 に は 心配 事 が ない 。
その 列車 は いつも 時刻 通り だ 。
2 時 に 出発 し たら 、 6 時 に は つく はず だ 。


## ここまで訓練
### ここからlocalで実行するために必要なものをインストールする

In [6]:
pip install fairseq==0.10.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U sacremoses

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade --user numpy

Requirement already up-to-date: numpy in c:\local\anaconda3\lib\site-packages (1.22.2)
Note: you may need to restart the kernel to use updated packages.


# 実践(localで)

In [1]:
import re
import unicodedata
from sacremoses import MosesTokenizer
import sentencepiece as spm
from fairseq.models.transformer import TransformerModel

In [2]:
mt = MosesTokenizer(lang = 'en')
sp = spm.SentencePieceProcessor(model_file='bpe.model')
ejmodel = TransformerModel.from_pretrained('./checkpoints/', checkpoint_file='checkpoint10.pt', data_name_or_path='data-bin')

def preproc_en(x):
  x = unicodedata.normalize('NFKC', x)
  x = re.sub(mt.AGGRESSIVE_HYPHEN_SPLIT[0], r'\1 - ', x)
  x = mt.tokenize(x, escape = False)
  x = ' '.join(x)
  x = x.lower()
  x = ' '.join(sp.encode(x, out_type = 'str'))
  return x

def ejtranslate(x):
  x = preproc_en(x)
  x = ejmodel.translate(x, beam = 5, lenpen = 0.6)
  x = ''.join(x.split()).replace('▁', '').strip()
  return x

2022-03-02 15:29:10 | INFO | fairseq.file_utils | loading archive file ./checkpoints/
2022-03-02 15:29:10 | INFO | fairseq.file_utils | loading archive file data-bin
2022-03-02 15:29:10 | INFO | fairseq.tasks.translation | [en] dictionary: 3888 types
2022-03-02 15:29:10 | INFO | fairseq.tasks.translation | [ja] dictionary: 3888 types
2022-03-02 15:29:11 | INFO | fairseq.models.fairseq_model | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin', data_buffer_size=10, dataset_impl=None, ddp

In [8]:
while True:
  x = input('英文を入力 > ')
  if not x:
    break
  x = ejtranslate(x)
  print('翻訳結果 > ' + x)

英文を入力 > 


In [6]:
from fairseq.models.transformer import TransformerModel

In [7]:
jemodel = TransformerModel.from_pretrained('./save98_2/', checkpoint_file='checkpoint10.pt', data_name_or_path='data98_2')

2022-03-02 15:29:29 | INFO | fairseq.file_utils | loading archive file ./save98_2/
2022-03-02 15:29:29 | INFO | fairseq.file_utils | loading archive file data98_2
2022-03-02 15:29:30 | INFO | fairseq.tasks.translation | [ja] dictionary: 31144 types
2022-03-02 15:29:30 | INFO | fairseq.tasks.translation | [en] dictionary: 29424 types
2022-03-02 15:29:30 | INFO | fairseq.models.fairseq_model | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data98_2', data_buffer_size=10, dataset_impl=None, ddp_

In [9]:
import MeCab
import sys
import re 

In [10]:
def jetranslate(x):
    wakati = MeCab.Tagger('-Owakati')
    wakati = ' '.join(wakati.parse(x).split())
    x = jemodel.translate(wakati, beam = 5, lenpen = 0.6) 
    x = re.sub('@@ ?$', '', x)
    x = re.sub('@@ ', '', x)
    return x

In [11]:
text = "私は、道元です。"
print(jetranslate(text))

I've been adopted by Dogen.


In [12]:
from flask import Flask, render_template, request

# WEBサーバ　翻訳機
jupyter notebookでプログラム実行後

`ngrok http 5000`

をpowershellなどで実行すると公開できる
https://3668-58-190-38-21.ngrok.io

In [ ]:
app = Flask(__name__)

@app.route("/", methods=["GET"])
def get():
    return render_template("index.html", \
        title = "日⇔英翻訳機", \
        message = "Please enter the text you want to translate here.(翻訳したい文を入れてください)",\
        ejmessage = "",\
        jemessage = "")

@app.route("/", methods=["POST"])
def post():
    ejtext = request.form["ejname"]
    #if re.compile(\s*).search(ejtext):
    if ejtext =="":
     ejoutput =""
    else:
     ejoutput = ejtranslate(ejtext)
    jetext = request.form["jename"]
    if jetext =="":
     jeoutput =""
    else:    
     jeoutput = jetranslate(jetext)
    return render_template("index.html",\
        title = "日⇔英翻訳機(出力結果)",\
        message = "翻訳完了",\
        ejmessage = "英→日結果:　　"+ejtext+"　　→　　"+ejoutput ,\
        jemessage = "日→英結果:　　"+jetext+"　　→　　"+jeoutput)
        

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2022-03-02 15:29:44 | INFO | werkzeug |  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
2022-03-02 15:29:48 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:29:48] "GET / HTTP/1.1" 200 -
2022-03-02 15:29:48 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:29:48] "GET /favicon.ico HTTP/1.1" 404 -
2022-03-02 15:31:18 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:31:18] "GET / HTTP/1.1" 200 -
2022-03-02 15:32:11 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:32:11] "GET / HTTP/1.1" 200 -
2022-03-02 15:32:12 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:32:12] "GET /favicon.ico HTTP/1.1" 404 -
2022-03-02 15:32:17 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:32:17] "POST / HTTP/1.1" 200 -
2022-03-02 15:32:23 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:32:23] "POST / HTTP/1.1" 200 -
2022-03-02 15:32:29 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:32:29] "POST / HTTP/1.1" 200 -
2022-03-02 15:33:51 | INFO | werkzeug | 127.0.0.1 - - [02/Mar/2022 15:33:51

## test用

In [14]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
  return 'Hello World!'

if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2022-02-28 12:18:42 | INFO | werkzeug |  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
2022-02-28 12:18:44 | ERROR | __main__ | Exception on / [GET]
Traceback (most recent call last):
  File "C:\local\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\local\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\local\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\local\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\local\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\local\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-13